# Chapter 09: -3- UnitTable.py and the unit_tables directory
2020-11-17

Content:
1. UML diagram of the unitconver package
2. setup.py and CLI.py for the entry point (and argParse module)
3. **UnitTable.py and the unit_tables directory (and lambda function)**
4. Converter.py (and importlib module)
5. Putting all of them together!

## 幾個重要的概念：
1. lambda function: 同一名稱的 function 可透過一個「樣式」的調整參數，而有不同的面貌
2. **import importlib**: 如何載入一個 py 檔的 module
3. **getattr()** 如何從一個 module 中，取出其中定義的一個 class 的 type，然後利用在 **type 後馬上補上()** 就可 instantiate 一個相對應的物件

**以上 3. 和 4. 是 run-time 實現：**
```python:
from unitconverter.unit_tables.time import time
a_time_table = time()
```
4. 取得物件之後，會將其定義在這物件中，內藏的 lambda function 給取出來作運算。

## -3- 先看基礎物件的 UnitTable.py
1. 在 unit_tables 資料夾內，含定義了各種不同單位類型的類別，所有的這些物件都繼承了UnitTable 這個 class
2. 在class UnitTable中有下列這些 attributes:
   + self.base_unit = None
   + self.to_base_unit = {}   # 字典
   + self.from_base_unit = {} # 字典
3. 在class UnitTable中有下列這些 methods：
    + convert: 用於單位轉換
    + get_units: 回傳所有支援的單位名稱

In [1]:
import os
course_directory = 'D:\\Google雲端硬碟\\GettingStartedWithPythonAndRaspberryPi-book_release'
os.chdir(course_directory + "/Chapter09/unitconverter")

In [2]:
!unitconvert time list

Unit table time can convert between the units:
m
ms
h
us
ns
y
s
d


In [3]:
!unitconvert time list -m

Unit table time can convert between the units:
ms (x * 1000.0)
y (x / 3.15569e7)
d (x / 86400.0)
ns (x * 1000000000.0)
s (base unit)
us (x * 1000000.0)
h (x / 3600.0)
m (x / 60.0)


In [4]:
!unitconvert time convert 300 d y ms h s

300.000000 d = 0.821373 y
300.000000 d = 25920000000.000000 ms
300.000000 d = 7200.000000 h
300.000000 d = 25920000.000000 s


In [5]:
! unitconvert energy convert 1000 j wh kcal hph

1000.000000 j = 0.277778 wh
1000.000000 j = 0.238846 kcal
1000.000000 j = 0.000373 hph


In [ ]:
'''
再來看 UnitTable.py 中，
所有要轉換的 Table 都是繼承這個 UnitTable 
'''
class UnitTable(object):
    """
    A class to manage conversion of units for the same quantity (e.g. length,
    volume, mass)
    """

    def __init__(self):
        self.base_unit = None
        self.to_base_unit = {}   # 字典
        self.from_base_unit = {} # 字典


    def convert(self, source, dest, value):
        """
        Converts a value in one unit to another.

        @param source The source unit
        @param dest The destination unit
        @param value The value to convert
        @return The value in the destination unit
        """

        if not (self.can_convert(source) and self.can_convert(dest)):
            raise ValueError('Cannot convert given units')

        if source != self.base_unit:
            value = self.to_base_unit[source](value)

        if dest != self.base_unit:
            value = self.from_base_unit[dest](value)

        return value


    def get_units(self):
        """
        Returns a list of units that this table can convert.

        @return List of valid units
        """

        convertable_units = set(self.to_base_unit.keys()) & set(self.from_base_unit.keys())
        convertable_units.add(self.base_unit)
        return convertable_units
    
    def can_convert(self, unit):
        """
        Checks to see if this table can convert a given unit.

        @param unit Name of unit to check
        @return Boolean indicating if this unit can be converted
        """

        return unit.lower() in self.get_units()


In [6]:
from unitconverter.UnitTable import UnitTable

In [7]:
help(UnitTable)

Help on class UnitTable in module unitconverter.UnitTable:

class UnitTable(builtins.object)
 |  A class to manage conversion of units for the same quantity (e.g. length,
 |  volume, mass)
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  can_convert(self, unit)
 |      Checks to see if this table can convert a given unit.
 |      
 |      @param unit Name of unit to check
 |      @return Boolean indicating if this unit can be converted
 |  
 |  convert(self, source, dest, value)
 |      Converts a value in one unit to another.
 |      
 |      @param source The source unit
 |      @param dest The destination unit
 |      @param value The value to convert
 |      @return The value in the destination unit
 |  
 |  get_units(self)
 |      Returns a list of units that this table can convert.
 |      
 |      @return List of valid units
 |  
 |  -------------------------------------------------------------

## 不同單位間的轉換：source  → destination 的計算流程：
中間有 base 當橋接的流程：
1. {source}  → to_base_unit → {base_unit} 
2. {base_unit} → from_base_unit → {destination} 

我們看一下在 unit_tables 底下各個 table 中所定義的兩種  lamda function
1. to_base_unit
2. from_base_unit

In [8]:
'''
當作繼承 UnitTable 的子輩，以下是 time.py 的內容
'''
from unitconverter import UnitTable

class time(UnitTable.UnitTable): ## 繼承 UnitTable

    def __init__(self):
        UnitTable.UnitTable.__init__(self)

        
        # Base unit is second
        self.base_unit = 's'

        # Minute
        self.to_base_unit['m']     = lambda x: x * 60.0
        self.from_base_unit['m']   = lambda x: x / 60.0
        
        #  使用時的語法：     value = self.to_base_unit[source](value)
        #  to_ 與 from_ 為倒數關係

        # Hour
        self.to_base_unit['h']     = lambda x: x * 3600.0
        self.from_base_unit['h']   = lambda x: x / 3600.0

        # Day
        self.to_base_unit['d']     = lambda x: x * 86400.0
        self.from_base_unit['d']   = lambda x: x / 86400.0

        # Year
        self.to_base_unit['y']     = lambda x: x * 3.15569e7
        self.from_base_unit['y']   = lambda x: x / 3.15569e7

        # Millisecond
        self.to_base_unit['ms']    = lambda x: x / 1000.0
        self.from_base_unit['ms']  = lambda x: x * 1000.0

        # Microsecond
        self.to_base_unit['us']    = lambda x: x / 1000000.0
        self.from_base_unit['us']  = lambda x: x * 1000000.0

        # Nanosecond
        self.to_base_unit['ns']    = lambda x: x / 1000000000.0
        self.from_base_unit['ns']  = lambda x: x * 1000000000.0


In [9]:
from unitconverter.unit_tables.time import time

In [10]:
help(time)

Help on class time in module unitconverter.unit_tables.time:

class time(unitconverter.UnitTable.UnitTable)
 |  A class to manage conversion of units for the same quantity (e.g. length,
 |  volume, mass)
 |  
 |  Method resolution order:
 |      time
 |      unitconverter.UnitTable.UnitTable
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from unitconverter.UnitTable.UnitTable:
 |  
 |  can_convert(self, unit)
 |      Checks to see if this table can convert a given unit.
 |      
 |      @param unit Name of unit to check
 |      @return Boolean indicating if this unit can be converted
 |  
 |  convert(self, source, dest, value)
 |      Converts a value in one unit to another.
 |      
 |      @param source The source unit
 |      @param dest The destination unit
 |      @param value Th

In [24]:
time

unitconverter.unit_tables.time.time

In [25]:
type(time)

type

In [11]:
a_time_table = time()

## 一旦取得這樣的物件之後，就可以看它的 attributes 以及試它的 methods

In [12]:
a_time_table.base_unit

's'

In [13]:
a_time_table.to_base_unit

{'m': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
 'h': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
 'd': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
 'y': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
 'ms': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
 'us': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
 'ns': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>}

In [14]:
a_time_table.to_base_unit['m']

<function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>

In [15]:
a_time_table.to_base_unit['m'](1)

60.0

In [16]:
a_time_table.from_base_unit

{'m': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
 'h': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
 'd': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
 'y': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
 'ms': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
 'us': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
 'ns': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>}

In [17]:
a_time_table.from_base_unit["m"]

<function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>

In [18]:
a_time_table.from_base_unit["m"](1)

0.016666666666666666

In [19]:
a_time_table.get_units()

{'d', 'h', 'm', 'ms', 'ns', 's', 'us', 'y'}

In [20]:
a_time_table.convert("y", "d", 1)

365.24189814814815

In [21]:
a_time_table.can_convert("m")

True

In [22]:
%whos

Variable           Type      Data/Info
--------------------------------------
UnitTable          module    <module 'unitconverter.Un<...>converter\\UnitTable.py'>
a_time_table       time      <unitconverter.unit_table<...>ct at 0x00000216D8FCDA20>
course_directory   str       D:\Google雲端硬碟\GettingStar<...>dRaspberryPi-book_release
os                 module    <module 'os' from 'C:\\Us<...>\\Anaconda3\\lib\\os.py'>
time               type      <class 'unitconverter.unit_tables.time.time'>


In [26]:
# 我這邊作了一個小實驗，把 time 給覆寫掉…
time = time()

In [27]:
# 注意看 time 的轉變
%whos

Variable           Type      Data/Info
--------------------------------------
UnitTable          module    <module 'unitconverter.Un<...>converter\\UnitTable.py'>
a_time_table       time      <unitconverter.unit_table<...>ct at 0x00000216D8FCDA20>
course_directory   str       D:\Google雲端硬碟\GettingStar<...>dRaspberryPi-book_release
os                 module    <module 'os' from 'C:\\Us<...>\\Anaconda3\\lib\\os.py'>
time               time      <unitconverter.unit_table<...>ct at 0x00000216D8FDE240>


## 使用 importlib 的實際考量：
unitconverter 在執行時，我們不並知道會被點到那個一 table，\
如果預先先建好各類的 table，恐怕太浪費。\

**與其先 import 好所有的 modules, \
？有沒有可能 run-time import 我們真正的要 module?** 

所以會想要，在 run-time，針對指名的 table，再 run-time 去建相對應的物件，\
然後就可以從這 table 的物件中，取得相關的 lambda function 等資訊。

當然，這需要一開始建置的各 table 的命名，先配合好。\
也就是說，當要作 時間轉換時，我就得先將相關的所有資訊存在一個叫作 time.py 的模組，\
**並且要放在指定有 unit_tables 目錄底下，還有 module 名稱。**

In [28]:
"""
run-time import 我所需要的 module

以下示例如何取得整個套件中，某個 module 裏的 class，然後 instantiate 一個物件
"""
import importlib
table_name = "time"
converter_module = importlib.import_module('unitconverter.unit_tables.%s' \
                                           % table_name)
# 等效為 import unitconverter.unit_tables.time
converter_module

<module 'unitconverter.unit_tables.time' from 'D:\\Google雲端硬碟\\GettingStartedWithPythonAndRaspberryPi-book_release\\Chapter09\\unitconverter\\unitconverter\\unit_tables\\time.py'>

In [29]:
%whos

Variable           Type      Data/Info
--------------------------------------
UnitTable          module    <module 'unitconverter.Un<...>converter\\UnitTable.py'>
a_time_table       time      <unitconverter.unit_table<...>ct at 0x00000216D8FCDA20>
converter_module   module    <module 'unitconverter.un<...>r\\unit_tables\\time.py'>
course_directory   str       D:\Google雲端硬碟\GettingStar<...>dRaspberryPi-book_release
importlib          module    <module 'importlib' from <...>\importlib\\__init__.py'>
os                 module    <module 'os' from 'C:\\Us<...>\\Anaconda3\\lib\\os.py'>
table_name         str       time
time               time      <unitconverter.unit_table<...>ct at 0x00000216D8FDE240>


In [32]:
importlib.__dict__

{'__name__': 'importlib',
 '__doc__': 'A pure Python implementation of import.',
 '__package__': 'importlib',
 '__loader__': <_frozen_importlib_external.SourceFileLoader at 0x216d675fe48>,
 '__spec__': ModuleSpec(name='importlib', loader=<_frozen_importlib_external.SourceFileLoader object at 0x00000216D675FE48>, origin='C:\\Users\\Public\\Anaconda3\\lib\\importlib\\__init__.py', submodule_search_locations=['C:\\Users\\Public\\Anaconda3\\lib\\importlib']),
 '__path__': ['C:\\Users\\Public\\Anaconda3\\lib\\importlib'],
 '__file__': 'C:\\Users\\Public\\Anaconda3\\lib\\importlib\\__init__.py',
 '__cached__': 'C:\\Users\\Public\\Anaconda3\\lib\\importlib\\__pycache__\\__init__.cpython-37.pyc',
 '__builtins__': {'__name__': 'builtins',
  '__doc__': "Built-in functions, exceptions, and other objects.\n\nNoteworthy: None is the `nil' object; Ellipsis represents `...' in slices.",
  '__package__': '',
  '__loader__': _frozen_importlib.BuiltinImporter,
  '__spec__': ModuleSpec(name='builtins', l

In [33]:
type(converter_module)

module

In [34]:
converter_module.__dict__

{'__name__': 'unitconverter.unit_tables.time',
 '__doc__': None,
 '__package__': 'unitconverter.unit_tables',
 '__loader__': <_frozen_importlib_external.SourceFileLoader at 0x216d8fcd3c8>,
 '__spec__': ModuleSpec(name='unitconverter.unit_tables.time', loader=<_frozen_importlib_external.SourceFileLoader object at 0x00000216D8FCD3C8>, origin='D:\\Google雲端硬碟\\GettingStartedWithPythonAndRaspberryPi-book_release\\Chapter09\\unitconverter\\unitconverter\\unit_tables\\time.py'),
 '__file__': 'D:\\Google雲端硬碟\\GettingStartedWithPythonAndRaspberryPi-book_release\\Chapter09\\unitconverter\\unitconverter\\unit_tables\\time.py',
 '__cached__': 'D:\\Google雲端硬碟\\GettingStartedWithPythonAndRaspberryPi-book_release\\Chapter09\\unitconverter\\unitconverter\\unit_tables\\__pycache__\\time.cpython-37.pyc',
 '__builtins__': {'__name__': 'builtins',
  '__doc__': "Built-in functions, exceptions, and other objects.\n\nNoteworthy: None is the `nil' object; Ellipsis represents `...' in slices.",
  '__package__'

In [35]:
converter_module.__dict__.keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__file__', '__cached__', '__builtins__', 'UnitTable', 'time'])

In [36]:
# 1st 
converter_module.time

unitconverter.unit_tables.time.time

In [37]:
converter_module.UnitTable

<module 'unitconverter.UnitTable' from 'D:\\Google雲端硬碟\\GettingStartedWithPythonAndRaspberryPi-book_release\\Chapter09\\unitconverter\\unitconverter\\UnitTable.py'>

In [38]:
# 2nd methods
converter_module.__dict__["time"]

unitconverter.unit_tables.time.time

In [39]:
help(getattr)

Help on built-in function getattr in module builtins:

getattr(...)
    getattr(object, name[, default]) -> value
    
    Get a named attribute from an object; getattr(x, 'y') is equivalent to x.y.
    When a default argument is given, it is returned when the attribute doesn't
    exist; without it, an exception is raised in that case.



### importlib.import_module() 把指定 package.directory.module 的 .py 程式給找到

In [41]:
# 3rd methods
getattr(converter_module, "time")

unitconverter.unit_tables.time.time

In [43]:
converter_class = getattr(converter_module, table_name)
converter_class

unitconverter.unit_tables.time.time

In [44]:
c = converter_class()
c

### getattr() 再把裏面的 "time" 的 class 給翻出來，指的是：

unitconverter.unit_tables.time.time
- unitconverter package 底下
- unit_tables 目錄底下的
- time.py module 中，所定義的
- time class

In [45]:
type(converter_class)

type

In [46]:
from unitconverter.UnitTable import UnitTable

In [47]:
type(UnitTable)

type

### 神奇的是，它屬於 type 的資料型態，而且我們直接在它後面加上 ()，就會變出一個相對應的 object 來！

In [48]:
# 我們直接在它後面加上 ()
a = converter_class()
a

### 竟然就可以變出 (instantiate) 一個 time 的 object 來！

In [49]:
%whos

Variable           Type      Data/Info
--------------------------------------
UnitTable          type      <class 'unitconverter.UnitTable.UnitTable'>
a                  time      <unitconverter.unit_table<...>ct at 0x00000216DAC95F60>
a_time_table       time      <unitconverter.unit_table<...>ct at 0x00000216D8FCDA20>
c                  time      <unitconverter.unit_table<...>ct at 0x00000216DAC95D30>
converter_class    type      <class 'unitconverter.unit_tables.time.time'>
converter_module   module    <module 'unitconverter.un<...>r\\unit_tables\\time.py'>
course_directory   str       D:\Google雲端硬碟\GettingStar<...>dRaspberryPi-book_release
importlib          module    <module 'importlib' from <...>\importlib\\__init__.py'>
os                 module    <module 'os' from 'C:\\Us<...>\\Anaconda3\\lib\\os.py'>
table_name         str       time
time               time      <unitconverter.unit_table<...>ct at 0x00000216D8FDE240>


In [51]:
# 如果一開始寫死的話
from unitconverter.unit_tables.time import time
another_time_object = time()

In [52]:
# 課本中寫的指令是用 convert_class, 但我覺得如果後面用了 (), 應該是
# object 才對。
import importlib
table_name = "time"
converter_module = importlib.import_module('unitconverter.unit_tables.%s' \
                                           % table_name)
# 以下是 run-time instantiate 一個　object
converter_object = getattr(converter_module, table_name)()

In [53]:
converter_object

In [54]:
%whos

Variable              Type      Data/Info
-----------------------------------------
UnitTable             type      <class 'unitconverter.UnitTable.UnitTable'>
a                     time      <unitconverter.unit_table<...>ct at 0x00000216DAC95F60>
a_time_table          time      <unitconverter.unit_table<...>ct at 0x00000216D8FCDA20>
another_time_object   time      <unitconverter.unit_table<...>ct at 0x00000216DACBC048>
c                     time      <unitconverter.unit_table<...>ct at 0x00000216DAC95D30>
converter_class       type      <class 'unitconverter.unit_tables.time.time'>
converter_module      module    <module 'unitconverter.un<...>r\\unit_tables\\time.py'>
converter_object      time      <unitconverter.unit_table<...>ct at 0x00000216DACEAC88>
course_directory      str       D:\Google雲端硬碟\GettingStar<...>dRaspberryPi-book_release
importlib             module    <module 'importlib' from <...>\importlib\\__init__.py'>
os                    module    <module 'os' from 'C:\\Us<

In [55]:
converter_object.__dict__

{'base_unit': 's',
 'to_base_unit': {'m': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'h': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'd': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'y': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'ms': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'us': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'ns': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>},
 'from_base_unit': {'m': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'h': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'd': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'y': <function unitconverter.unit_tables.time.time.__init__.<locals>.<lambda>(x)>,
  'ms': 

In [56]:
type(converter_object)

unitconverter.unit_tables.time.time

In [57]:
converter_object

In [58]:
converter_object.base_unit

's'

In [59]:
converter_object.get_units()

{'d', 'h', 'm', 'ms', 'ns', 's', 'us', 'y'}